In [1]:
#중고차 가격 예측
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv")


display(x_train.head())
display(y_train.head())

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


,carID,price
0,13207,31995
1,17314,7700
2,12342,58990
3,13426,12999
4,16004,16990


In [8]:
x_train.drop(columns = {"carID"},inplace=True)

In [35]:
x = x_train.copy()
x.drop(columns = {'brand',"transmission","model","fuelType","year"},inplace=True)
x = x.join(pd.get_dummies(x_train[['brand','transmission','model','fuelType','year']]))
x['year'] = x['year'] - x['year'].min()
x

,mileage,tax,mpg,engineSize,year,brand_audi,brand_bmw,brand_ford,brand_hyundi,brand_merc,...,model_ Z3,model_ Z4,model_ Zafira Tourer,model_ i3,model_ i8,fuelType_Diesel,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol
0,4223,145.0,39.8,2.2,22,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,47870,125.0,60.1,2.0,18,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,5151,145.0,29.1,2.9,22,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,20423,30.0,57.6,2.0,19,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,3569,145.0,47.1,1.0,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4955,24314,125.0,56.6,2.1,18,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4956,18000,145.0,51.4,3.0,20,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4957,84932,30.0,60.1,2.0,18,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4958,30150,145.0,62.8,3.0,20,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [56]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error,r2_score
import numpy as np 
lin_model = LinearRegression()
kfold = KFold()
acc = []
r2 = []
for train_idx, test_idx in kfold.split(x):
    lin_model.fit(x.loc[train_idx],y_train.loc[train_idx]['price'])
    pred = lin_model.predict(x.loc[test_idx])
    acc.append(mean_absolute_error(pred,y_train.loc[test_idx]['price']))
    r2.append(r2_score(pred,y_train.loc[test_idx]['price']))
print(np.mean(r2))
print(np.mean(acc))

0.8656838861037522
3585.141859423681


In [57]:
from sklearn.ensemble import BaggingRegressor
lin_model = BaggingRegressor()
kfold = KFold()
acc = []
for train_idx, test_idx in kfold.split(x):
    lin_model.fit(x.loc[train_idx],y_train.loc[train_idx]['price'])
    pred = lin_model.predict(x.loc[test_idx])
    acc.append(mean_absolute_error(pred,y_train.loc[test_idx]['price']))
    r2.append(r2_score(pred,y_train.loc[test_idx]['price']))
print(np.mean(r2))
print(np.mean(acc))

0.9077486513267606
2065.6291057572016


In [115]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
minmax = MinMaxScaler()

minmax_df = x.copy()
minmax.fit(x_train[["mileage","tax","mpg","engineSize","year"]])
minmax_df[["mileage","tax","mpg","engineSize","year"]] = minmax.transform(x_train[["mileage","tax","mpg","engineSize","year"]])

lin_model = LinearRegression()
kfold = KFold()
acc = []
r2 = []
for train_idx, test_idx in kfold.split(minmax_df):
    lin_model.fit(minmax_df.loc[train_idx],y_train.loc[train_idx]['price'])
    pred = lin_model.predict(minmax_df.loc[test_idx])
    acc.append(mean_absolute_error(pred,y_train.loc[test_idx]['price']))
    r2.append(r2_score(pred,y_train.loc[test_idx]['price']))
print(np.mean(r2))
print(np.mean(acc))

0.6932884274138745
7534003718.707886


In [113]:
from sklearn.ensemble import BaggingRegressor
lin_model = BaggingRegressor()
kfold = KFold()
acc = []
for train_idx, test_idx in kfold.split(minmax_df):
    lin_model.fit(minmax_df.loc[train_idx],y_train.loc[train_idx]['price'])
    pred = lin_model.predict(minmax_df.loc[test_idx])
    acc.append(mean_absolute_error(pred,y_train.loc[test_idx]['price']))
    r2.append(r2_score(pred,y_train.loc[test_idx]['price']))
print(np.mean(r2))
print(np.mean(acc))

0.82005346338726
2035.1841144546702


In [116]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
std = StandardScaler()

std_df = x.copy()
std.fit(x_train[["mileage","tax","mpg","engineSize","year"]])
std_df[["mileage","tax","mpg","engineSize","year"]] = std.transform(x_train[["mileage","tax","mpg","engineSize","year"]])

lin_model = LinearRegression()
kfold = KFold()
acc = []
r2 = []
for train_idx, test_idx in kfold.split(std_df):
    lin_model.fit(std_df.loc[train_idx],y_train.loc[train_idx]['price'])
    pred = lin_model.predict(std_df.loc[test_idx])
    acc.append(mean_absolute_error(pred,y_train.loc[test_idx]['price']))
    r2.append(r2_score(pred,y_train.loc[test_idx]['price']))
print(np.mean(r2))
print(np.mean(acc))

from sklearn.ensemble import BaggingRegressor
lin_model = BaggingRegressor()
kfold = KFold()
acc = []
for train_idx, test_idx in kfold.split(std_df):
    lin_model.fit(std_df.loc[train_idx],y_train.loc[train_idx]['price'])
    pred = lin_model.predict(std_df.loc[test_idx])
    acc.append(mean_absolute_error(pred,y_train.loc[test_idx]['price']))
    r2.append(r2_score(pred,y_train.loc[test_idx]['price']))
print(np.mean(r2))
print(np.mean(acc))

0.6932725420729516
8223936565.753679
0.8199444390794337
2047.7584026079207


In [73]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
brand_encoder = LabelEncoder()
model_encoder = LabelEncoder()
fuel_encoder = LabelEncoder()
transmission_encoder = LabelEncoder()

brand_encoder.fit(x_train['brand'])
model_encoder.fit(x_train['model'])
fuel_encoder.fit(x_train['fuelType'])
transmission_encoder.fit(x_train['transmission'])

x_train['brand'] = brand_encoder.transform(x_train['brand'])
x_train['model'] = model_encoder.transform(x_train['model'])
x_train['fuelType'] = fuel_encoder.transform(x_train['fuelType'])
x_train['transmission'] = transmission_encoder.transform(x_train['transmission'])

In [91]:
from sklearn.model_selection import GridSearchCV
parameter = {"n_estimators" : [20,50,100], "max_depth": [5,8,10,13,15,20]}
model = RandomForestRegressor()
gcv = GridSearchCV(model,parameter,scoring="r2")
gcv.fit(x_train,y_train['price'])

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [5, 8, 10, 13, 15, 20],
                         'n_estimators': [20, 50, 100]},
             scoring='r2')

In [92]:
gcv.best_score_

0.9541137506013081

In [99]:
parameter = {"n_estimators" : [20,50,100], "max_depth": [5,8,10,13,15,20]}
model = RandomForestRegressor()
gcv = GridSearchCV(model,parameter,scoring="neg_mean_absolute_error")
gcv.fit(x_train,y_train['price'])

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [5, 8, 10, 13, 15, 20],
                         'n_estimators': [20, 50, 100]},
             scoring='neg_mean_absolute_error')

In [101]:
print(gcv.best_score_)

-1975.1328302640045


In [7]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2,8651400730,20150428T000000,3,1.00,840,5525,1.0,0,0,...,6,840,0,1969,0,98042,47.3607,-122.085,920,5330
1,3,3163600130,20150317T000000,3,1.00,1250,8000,1.0,0,0,...,7,1250,0,1956,0,98146,47.5065,-122.337,1040,6973
2,4,5045700330,20140725T000000,4,2.50,2200,6400,2.0,0,0,...,8,2200,0,2010,0,98059,47.4856,-122.156,2600,5870
3,5,1036100130,20140808T000000,3,2.50,1980,39932,2.0,0,0,...,8,1980,0,1994,0,98011,47.7433,-122.196,2610,12769
4,6,7696630080,20140506T000000,3,1.75,1690,7735,1.0,0,0,...,7,1060,630,1976,0,98001,47.3324,-122.280,1580,7503


,ID,price
0,2,191000.0
1,3,234900.0
2,4,460000.0
3,5,442000.0
4,6,197000.0


In [9]:
x_train.drop(columns = {"id","ID",'date'},inplace= True)


In [15]:
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.model_selection import GridSearchCV

parameter = {"max_depth" : [5,10,13,15,20],"n_estimators":[20,50]}

model = RandomForestRegressor()
gscv = GridSearchCV(model,parameter,scoring="r2")
gscv.fit(x_train,y_train['price'])
print(gscv.best_score_)



0.8780791885761273


ValueError: 'neg_mean_absoulte_error' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.

In [18]:
parameter = {"max_depth" : [5,10,13,15,20],"n_estimators":[20,50]}

gscv = GridSearchCV(model,parameter,scoring="neg_mean_absolute_error")
gscv.fit(x_train,y_train['price'])
print(gscv.best_score_)

-70865.34905804152


In [22]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_absolute_error
kfold = KFold()
model = BaggingRegressor()
r2 = []
mae = []
for train_idx,test_idx in kfold.split(x_train):
    model.fit(x_train.iloc[train_idx],y_train.iloc[train_idx]['price'])
    pred = model.predict(x_train.iloc[test_idx])
    r2.append(r2_score(y_train.iloc[test_idx]['price'],pred))
    mae.append(mean_absolute_error(y_train.iloc[test_idx]['price'],pred))
    
import numpy as np
print(np.mean(r2))
print(np.mean(mae))

0.8632847892895933
74500.8957060233


In [24]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_absolute_error
kfold = KFold()
model = BaggingRegressor()
r2 = []
mae = []
for train_idx,test_idx in kfold.split(x_train):
    model.fit(x_train.iloc[train_idx],y_train.iloc[train_idx]['price'])
    pred = model.predict(x_train.iloc[test_idx])
    r2.append(r2_score(y_train.iloc[test_idx]['price'],pred))
    mae.append(mean_absolute_error(y_train.iloc[test_idx]['price'],pred))
    
import numpy as np
print(np.mean(r2))
print(np.mean(mae))

count    1.729000e+04
mean     5.415325e+05
std      3.740939e+05
min      7.800000e+04
25%      3.206750e+05
50%      4.500000e+05
75%      6.450000e+05
max      7.700000e+06
Name: price, dtype: float64

In [34]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
mm = MinMaxScaler()
mm.fit(x_train)
x = x_train.copy()
x[x.columns]= mm.transform(x_train)

kfold = KFold()
model = BaggingRegressor()
r2 = []
mae = []
for train_idx,test_idx in kfold.split(x):
    model.fit(x.iloc[train_idx],y_train.iloc[train_idx]['price'])
    pred = model.predict(x.iloc[test_idx])
    r2.append(r2_score(y_train.iloc[test_idx]['price'],pred))
    mae.append(mean_absolute_error(y_train.iloc[test_idx]['price'],pred))
    
import numpy as np
print(np.mean(r2))
print(np.mean(mae))

0.8700004256075065
73976.05663341044


In [35]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
mm = StandardScaler()
mm.fit(x_train)
x = x_train.copy()
x[x.columns]= mm.transform(x_train)

kfold = KFold()
model = BaggingRegressor()
r2 = []
mae = []
for train_idx,test_idx in kfold.split(x):
    model.fit(x.iloc[train_idx],y_train.iloc[train_idx]['price'])
    pred = model.predict(x.iloc[test_idx])
    r2.append(r2_score(y_train.iloc[test_idx]['price'],pred))
    mae.append(mean_absolute_error(y_train.iloc[test_idx]['price'],pred))
    
import numpy as np
print(np.mean(r2))
print(np.mean(mae))

0.8646146385138778
74273.07999823734


In [37]:
x_train.isnull().sum()

bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64